# 简介

恭喜你一路来到复赛！如今你已经是 IPEX-LLM 工具链的使用专家了；得益于 Intel 工具链对大语言模型推理的优化，如今我们可以让 LLM 应用在 Intel CPU 上获得强大的推理性能。在本次复赛中，我们将使用性能更加强大的 g8i 实例，我们将在搭载了英特尔最新第四代英特尔® 至强®可扩展处理器上运行更大参数量的大语言模型甚至扩散模型、多模态模型。
你是不是在初赛环节觉得有些意犹未尽？让我们在复赛环节真正的大展身手，共同感受 CPU 推理的极致性能体验。

首先，我们需要前往阿里云进入服务器界面，我们需要依次选择云服务器ECS——立即购买——按量付费；
随后，你需要完成以下操作：
1. 在第3页找到通用型g8i `ecs.g8i.6xlarge  24vCPU` 对应型号，镜像选择 Ubuntu 22.04 64位
2. 找到带宽和安全组，点选公网IP中的 `分配公网IPv4地址`
3. 选择`自定义密码`并设置密码后即可确认下单（价格约一小时6.321元）

至此，服务器实例创建完毕，你也拥有了一个可以公网访问的IP地址。（记得密码要写的复杂！否则容易被攻击，建议大小写特殊符号都要有）

> **❗ 重要信息**：如果您不会持续使用相关服务器实例，可以考虑下列方案,在保存现有代码和模型等数据的同事，节省费用支出：
>
> 请在实例界面选择停止实例后选择 **节省停机模式** ，待下次进入时可正常恢复开发环境，同时节约计费；如果你想完全停止所有实例计费，你需要在`更多操作`中完全释放实例，若仍是担心费用问题，可在左侧的 **块存储（云盘）** 处检查硬盘资源是否成功释放。


注意，在创建实例后，推荐在正式进入服务器之前，先进行存储的扩容。我们可以点击左侧`存储与快照`下的`块存储（云盘）`，看到当前有个使用中的系统盘，随后找到操作中的`扩容`，将其扩充至200~400G即可。

请参加比赛的各团队伙伴根据项目实际带宽及流量需求，综合考虑后选择合适的流量/带宽计费方案，可以根据实际情况需求进行动态修改。


# 一、安装环境

创建实例后，点击远程连接即可进入机器，但此时不方便操作，我们可以通过 vscode ssh 远程连接到服务器实例，通过密码验证直接登录 `ssh root@xxx.xxx.xxx.xxx`。进入机器后，我们需要安装最新的 IPEX-LLM 程序，你也可以直接把这个notebook移动到服务器上 `/home` 目录下进行操作。vscode需要安装python和jupyter依赖等。

> 我们并不需要拘泥于当前的 IPEX-LLM 技术方案，欢迎大家使用各种比赛规则中推荐的部署方案进行项目实现。    
>
> 本notebook仅提供一种部署及实现RAG的参考方案，更多方案、以及资料请参考：   
> - [OpenVINO LLMs](https://docs.llamaindex.ai/en/stable/examples/llm/openvino/)  
> - [llm-rag-langchain-with-output](https://docs.openvino.ai/nightly/notebooks/llm-rag-langchain-with-output.html)
> - [llm-rag-llamaindex-with-output](https://docs.openvino.ai/nightly/notebooks/llm-rag-llamaindex-with-output.html)
> - [Intel® Extension for Transformers](https://github.com/intel/intel-extension-for-transformers/blob/main/docs/weightonlyquant.md#examples-for-gpu)
> - [xFasterTransformer](https://github.com/intel/xFasterTransformer)
> - [Intel Extension for Pytorch](https://github.com/intel/intel-extension-for-pytorch)

In [ ]:
!pip install modelscope
!pip install ipex-llm==2.1.0b20240805
!pip install transformers==4.37.0 accelerate
!pip install PyMuPDF llama-index-vector-stores-chroma llama-index-readers-file llama-index-embeddings-huggingface llama-index
!pip install py-cpuinfo
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cpu

# 二、模型准备

Qwen2是阿里云最新推出的开源大型语言模型系列，相比Qwen1.5，Qwen2实现了整体性能的代际飞跃，大幅提升了代码、数学、推理、指令遵循、多语言理解等能力。

包含5个尺寸的预训练和指令微调模型：Qwen2-0.5B、Qwen2-1.5B、Qwen2-7B、Qwen2-57B-A14B和Qwen2-72B，其中Qwen2-57B-A14B为混合专家模型（MoE）。所有尺寸模型都使用了GQA（分组查询注意力）机制，以便让用户体验到GQA带来的推理加速和显存占用降低的优势。

在中文、英语的基础上，训练数据中增加了27种语言相关的高质量数据。增大了上下文长度支持，最高达到128K tokens（Qwen2-72B-Instruct）。

在这里，我们将使用 `Qwen/Qwen2-7B-Instruct` 的模型参数版本来体验 Qwen2 的强大能力。

首先，我们需要对模型进行下载，我们可以通过 modelscope 的 api 很容易实现模型的下载：


In [ ]:
from modelscope import snapshot_download
import os
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('Qwen/Qwen2-7B-Instruct', cache_dir='/home/qwen2chat_src', revision='master')

下载完成后，我们将对 qwen2 模型进行低精度量化至 int4 ，低精度量化（Low Precision Quantization）是指将浮点数转换为低位宽的整数（这里是int4），以减少计算资源的需求和提高系统的效率。这种技术在深度学习模型中尤其重要，它可以在硬件上实现快速、低功耗的推理，也可以加快模型加载的速度。

经过 Intel ipex-llm 优化后的大模型加载 api `from ipex_llm.transformers import AutoModelForCausalLM`， 我们可以很容易通过 `load_in_low_bit='sym_int4'` 将模型量化到 int4 ，英特尔 IPEX-LLM 支持 ‘sym_int4’, ‘asym_int4’, ‘sym_int5’, ‘asym_int5’ 或 'sym_int8’选项，其中 ‘sym’ 和 ‘asym’ 用于区分对称量化与非对称量化。 最后，我们将使用 `save_low_bit` api 将转换后的模型权重保存到指定文件夹。


In [ ]:
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import  AutoTokenizer
import os
if __name__ == '__main__':
    model_path = os.path.join(os.getcwd(),"/home/qwen2chat_src/Qwen/Qwen2-7B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(model_path, load_in_low_bit='sym_int4', trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model.save_low_bit('/home/qwen2chat_int4')
    tokenizer.save_pretrained('/home/qwen2chat_int4')
    print("保存完毕！")

准备完转换后的量化权重，接下来我们将在终端中第一次运行 qwen2 在 CPU 上的大模型推理，但请注意不要在 notebook 中运行（本地运行可以在 notebook 中运行，由于魔搭 notebook 和终端运行脚本有一些区别，这里推荐在终端中运行。

在运行下列代码块后，将会自动在终端中新建一个python文件，我们只需要在终端运行这个python文件即可启动推理：

```python
cd /home
python3 run_stream.py
```

In [ ]:
%%writefile /home/run_stream.py
# 设置OpenMP线程数为8
import os

import time
from transformers import AutoTokenizer
from transformers import TextStreamer

# 导入Intel扩展的Transformers模型
from ipex_llm.transformers import AutoModelForCausalLM
import torch

# 加载模型路径
load_path = "qwen2chat_int4"

# 加载4位量化的模型
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)

# 加载对应的tokenizer
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)

# 创建文本流式输出器
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# 设置提示词
prompt = "给我讲一个芯片制造的流程"

# 构建消息列表
messages = [{"role": "user", "content": prompt}]
    
# 使用推理模式
with torch.inference_mode():

    # 应用聊天模板,添加生成提示
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 对输入文本进行编码
    model_inputs = tokenizer([text], return_tensors="pt")
    
    print("start generate")
    st = time.time()  # 记录开始时间
    
    # 生成文本
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,  # 最大生成512个新token
        streamer=streamer,   # 使用流式输出
    )
    
    end = time.time()  # 记录结束时间
    
    # 打印推理时间
    print(f'Inference time: {end-st} s')


# 三、在 g8i 上实现 RAG 模块

在之前教程中,我们已经学会了如何应用 IPEX-LLM 加速推理,也学会了如何快速搭建一个基于 IPEX-LLM 的 RAG 系统.

让我们再来复习一下。LlamaIndex主要包括以下几个组件:
- 数据连接器：帮助连接现有数据源和数据格式（如API、PDF等），并将这些数据转换为LlamaIndex可用的格式。
- 数据索引：帮助结构化数据以适应不同的用例。加载了来自不同数据源的数据后，如何将它们分割、定义关系和组织，以便无论您想要解决的问题（问答、摘要等），都可以使用索引来检索相关信息。
- 查询接口：是输入查询并从LLM中获取知识增强输出的接口。

我们可以来看一个简单的 LlamaIndex 示例,它直观展示了如何构建一个 RAG 体系:

假设你有如下的文件组织:
```
├── starter.py
└── data
    └── paul_graham_essay.txt
```

核心代码为:
```python
# 导入需要的模块和类
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# 1. 使用 SimpleDirectoryReader 加载数据
# SimpleDirectoryReader 是一个简单的目录读取器，能从指定目录中读取所有文件的数据
documents = SimpleDirectoryReader("data").load_data()

# 2. 设置嵌入模型为 bge-base
# HuggingFaceEmbedding 是一个嵌入模型类，用于将文本转换为向量表示
# 这里我们使用的是 "BAAI/bge-base-en-v1.5" 模型
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# 3. 使用 Ollama 快速接入大语言模型
# Ollama 是一个模型的快速调用框架
# 这里我们指定使用 "llama3" 模型，并设置请求超时时间为 360 秒
Settings.llm = Ollama(model="llama3", request_timeout=360.0)

# 4. 创建一个向量存储索引
# VectorStoreIndex 是一个用于存储和查询向量的索引类
# from_documents 方法是从文档数据创建索引
index = VectorStoreIndex.from_documents(documents)

# 5. 将索引转换为查询引擎
# as_query_engine 方法将现有的向量存储索引转换为一个查询引擎
# 查询引擎用来对存储的数据进行语义查询
query_engine = index.as_query_engine()

# 6. 使用查询引擎进行查询
# query 方法接受一个查询字符串，并返回一个响应对象
# 这里我们查询 "作者小时候做了什么？"
response = query_engine.query("What did the author do growing up?")

# 7. 打印查询结果
# 打印从查询引擎返回的响应
print(response)
```

从代码中,我们可以很容易看到 RAG 系统构建过程,首先我们需要一个读取器来获得某个目录的对应数据,接着需要对这个数据进行 embedding 化即创建索引,把他转为向量表示,最后就可以用设定好的大模型,结合 query 与进行检索增强生成的对话.

接下来,我们将基于 Llamaindex 正式构建一个简易的 RAG 系统,首先我们需要下载中文 Embedding 模型并安装 RAG 系统所需的全部依赖,在这里我们将使用 pdf 文件作为示范.

In [ ]:
from modelscope import snapshot_download
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('AI-ModelScope/bge-small-zh-v1.5', cache_dir='/home/qwen2chat_src', revision='master')

除了上述推荐的 Embedding 模型以外，我们还可以尝试[更大参数](https://huggingface.co/BAAI/bge-large-zh-v1.5)的 Embedding 模型，BGE 是北京智源人工智能研究院开源的嵌入模型家族，你可以使用但不限于以下几类模型：

```
BAAI/bge-m3
BAAI/bge-large-zh-v1.5
BAAI/bge-base-zh-v1.5
BAAI/bge-small-zh-v1.5
```

你可以在[huggingface官网](https://huggingface.co/collections/BAAI/bge-66797a74476eb1f085c7446d)寻找到更多讯息，对于不同语言的初始文本，我们也推荐使用不同语言的嵌入模型进行调试，以便最大程度提升向量化的成效。除此之外，我们还推荐使用 [rerank 模型](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/llm_reranker) 进行检索重排操作，这会为检索结果带来进一步的改善，增强问答效果。


准备好向量化所需的 Embedding 模型后,我们还需要提前准备 RAG 使用的 pdf 文件进行向量化处理,你可以按照如下文件组织存放对应的 pdf 文件,你可以根据实际情况考虑是否提前构建好向量数据库,此时直接使用 LlamaIndex 再次加载即可使用.

```
├── run_rag.py
└── data
    └── 2407.10671v3.pdf
```

存放完 pdf 后,你需要修改下列 run_rag.py 代码块文件中的 `Config` 配置,将 question 修改成你想要对材料的提问,将 data_path 设定为 pdf 的文件地址,随后可以运行代码块创建出带运行的 RAG python 工程文件,紧接着按照下列方式即可启动:

```python
cd /home
python3 run_rag.py
```

In [ ]:
%%writefile /home/run_rag.py
import os
import time
import shutil

import torch
from typing import Any, List, Optional


# 从llama_index库导入HuggingFaceEmbedding类，用于将文本转换为向量表示
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# 从llama_index库导入ChromaVectorStore类，用于高效存储和检索向量数据
from llama_index.vector_stores.chroma import ChromaVectorStore
# 从llama_index库导入PyMuPDFReader类，用于读取和解析PDF文件内容
from llama_index.readers.file import PyMuPDFReader
# 从llama_index库导入NodeWithScore和TextNode类
# NodeWithScore: 表示带有相关性分数的节点，用于排序检索结果
# TextNode: 表示文本块，是索引和检索的基本单位。节点存储文本内容及其元数据，便于构建知识图谱和语义搜索
from llama_index.core.schema import NodeWithScore, TextNode
# 从llama_index库导入RetrieverQueryEngine类，用于协调检索器和响应生成，执行端到端的问答过程
from llama_index.core.query_engine import RetrieverQueryEngine
# 从llama_index库导入QueryBundle类，用于封装查询相关的信息，如查询文本、过滤器等
from llama_index.core import QueryBundle
# 从llama_index库导入BaseRetriever类，这是所有检索器的基类，定义了检索接口
from llama_index.core.retrievers import BaseRetriever
# 从llama_index库导入SentenceSplitter类，用于将长文本分割成句子或语义完整的文本块，便于索引和检索
from llama_index.core.node_parser import SentenceSplitter
# 从llama_index库导入VectorStoreQuery类，用于构造向量存储的查询，支持语义相似度搜索
from llama_index.core.vector_stores import VectorStoreQuery
# 向量数据库
import chromadb
from ipex_llm.llamaindex.llms import IpexLLM

class Config:
    """配置类,存储所有需要的参数"""
    model_path = "/home/qwen2chat_int4"
    tokenizer_path = "/home/qwen2chat_int4"
    question = "请你严格基于材料，告诉我qwen2的性能"
    data_path = "/home/data/2407.10671v3.pdf"
    persist_dir = "/home/chroma_db"
    embedding_model_path = "/home/qwen2chat_src/AI-ModelScope/bge-small-zh-v1___5"
    max_new_tokens = 2048
    if_persist_db = False # 决定是否要读取旧向量数据库，False 表示每次创建新向量数据库

def load_vector_database(if_persist_db: bool,persist_dir: str) -> ChromaVectorStore:
    """
    加载或创建向量数据库
    
    Args:
        persist_dir (str): 持久化目录路径
    
    Returns:
        ChromaVectorStore: 向量存储对象
    """
    # 检查持久化目录是否存在
    if if_persist_db:
        if os.path.exists(persist_dir):
            print(f"正在加载现有的向量数据库: {persist_dir}")
            chroma_client = chromadb.PersistentClient(path=persist_dir)
            chroma_collection = chroma_client.get_collection("qwen2_paper")
        else:
            print(f"创建新的向量数据库: {persist_dir}")
            chroma_client = chromadb.PersistentClient(path=persist_dir)
            chroma_collection = chroma_client.create_collection("qwen2_paper")
    else:
        if os.path.exists(persist_dir):
            print(f"发现重复向量数据库，进行删除: {persist_dir}")
            shutil.rmtree(persist_dir)
        print(f"创建新的向量数据库: {persist_dir}")
        chroma_client = chromadb.PersistentClient(path=persist_dir)
        chroma_collection = chroma_client.create_collection("qwen2_paper")
    print(f"Vector store loaded with {chroma_collection.count()} documents")
    return ChromaVectorStore(chroma_collection=chroma_collection)

def load_data(data_path: str) -> List[TextNode]:
    """
    加载并处理PDF数据
    
    Args:
        data_path (str): PDF文件路径
    
    Returns:
        List[TextNode]: 处理后的文本节点列表
    """
    loader = PyMuPDFReader()
    documents = loader.load(file_path=data_path)

    text_parser = SentenceSplitter(chunk_size=2048)
    text_chunks = []
    doc_idxs = []
    for doc_idx, doc in enumerate(documents):
        cur_text_chunks = text_parser.split_text(doc.text)
        text_chunks.extend(cur_text_chunks)
        doc_idxs.extend([doc_idx] * len(cur_text_chunks))

    nodes = []
    for idx, text_chunk in enumerate(text_chunks):
        node = TextNode(text=text_chunk)
        src_doc = documents[doc_idxs[idx]]
        node.metadata = src_doc.metadata
        nodes.append(node)
    return nodes

class VectorDBRetriever(BaseRetriever):
    """向量数据库检索器"""

    def __init__(
        self,
        vector_store: ChromaVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        检索相关文档
        
        Args:
            query_bundle (QueryBundle): 查询包
        
        Returns:
            List[NodeWithScore]: 检索到的文档节点及其相关性得分
        """
        query_embedding = self._embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = self._vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))
        print(f"Retrieved {len(nodes_with_scores)} nodes with scores")
        return nodes_with_scores

def completion_to_prompt(completion: str) -> str:
    """
    将完成转换为提示格式
    
    Args:
        completion (str): 完成的文本
    
    Returns:
        str: 格式化后的提示
    """
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

def messages_to_prompt(messages: List[dict]) -> str:
    """
    将消息列表转换为提示格式
    
    Args:
        messages (List[dict]): 消息列表
    
    Returns:
        str: 格式化后的提示
    """
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    prompt = prompt + "<|assistant|>\n"

    return prompt

def setup_llm(config: Config) -> IpexLLM:
    """
    设置语言模型
    
    Args:
        config (Config): 配置对象
    
    Returns:
        IpexLLM: 配置好的语言模型
    """
    return IpexLLM.from_model_id_low_bit(
        model_name=config.model_path,
        tokenizer_name=config.tokenizer_path,
        context_window=4096,
        max_new_tokens=config.max_new_tokens,
        generate_kwargs={"temperature": 0.5, "do_sample": True},
        model_kwargs={},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="cpu",
    )
def main():
    """主函数"""
    config = Config()
    
    # 设置嵌入模型
    embed_model = HuggingFaceEmbedding(model_name=config.embedding_model_path)
    
    # 设置语言模型
    llm = setup_llm(config)
    
    # 加载向量数据库
    vector_store = load_vector_database(if_persist_db=config.if_persist_db,persist_dir=config.persist_dir)
    
    # 加载和处理数据
    nodes = load_data(data_path=config.data_path)
    for node in nodes:
        node_embedding = embed_model.get_text_embedding(
            node.get_content(metadata_mode="all")
        )
        node.embedding = node_embedding
    
    # 将 node 添加到向量存储
    vector_store.add(nodes)
    
    # 设置查询
    query_str = config.question
    query_embedding = embed_model.get_query_embedding(query_str)
    
    # 执行向量存储检索
    print("开始执行向量存储检索")
    query_mode = "default"
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)

    # 处理查询结果
    print("开始处理检索结果")
    nodes_with_scores = []
    for index, node in enumerate(query_result.nodes):
        score: Optional[float] = None
        if query_result.similarities is not None:
            score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))
    
    # 设置检索器
    retriever = VectorDBRetriever(
        vector_store, embed_model, query_mode="default", similarity_top_k=1
    )
    
    print(f"Query engine created with retriever: {type(retriever).__name__}")
    print(f"Query string length: {len(query_str)}")
    print(f"Query string: {query_str}")
    
    # 创建查询引擎
    print("准备与llm对话")
    query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

    # 执行查询
    print("开始RAG最后生成")
    start_time = time.time()
    response = query_engine.query(query_str)

    # 打印结果
    print("------------RESPONSE GENERATION---------------------")
    print(str(response))
    print(f"inference time: {time.time()-start_time}")

if __name__ == "__main__":
    main()


当你能够看到正常的返回值,则证明你已经顺利实现了简单的 RAG 问答系统,接下来你可以尝试修改这个系统,或是用其他的 RAG 实现方法,来增强当前的问答体验.

如果你已经通过了初赛，相信这些操作对你来说完全不难，大家可以尝试在当前机器上运行甚至20B以上参数量的模型；当然，最关键的还是内容是否具有创新性，但我们相信对你来说这些已不再是难题。

祝你好运!